# Análise de Churn de Clientes da Telecom X

## Introdução

Este relatório detalha o processo de **Extração, Transformação e Carregamento (ETL)** e a **Análise Exploratória de Dados (EDA)** realizados sobre os dados de clientes da empresa de telecomunicações **Telecom X**. O objetivo principal é identificar os fatores que mais contribuem para a evasão de clientes (churn) e fornecer insights que possam auxiliar na criação de estratégias para retenção.

A alta taxa de churn é um problema crítico para empresas de serviços por assinatura, pois o custo de adquirir um novo cliente é geralmente muito superior ao de manter um existente. Através da análise dos dados, buscamos entender os padrões de comportamento dos clientes que cancelam seus serviços e, com isso, fornecer uma base sólida para que a equipe de Data Science possa desenvolver modelos preditivos eficazes.


## 1. Processo de Extração, Transformação e Carregamento (ETL)

O processo de ETL é a fundação de qualquer projeto de análise de dados. Nesta seção, descrevemos as etapas realizadas para obter, limpar e preparar os dados para a análise.


In [ ]:
import pandas as pd
import requests
import json

# A URL da API foi descoberta após uma busca em repositórios do GitHub relacionados ao desafio
url = "https://raw.githubusercontent.com/Lacerdash/Alura_Challenge_Data_Science/refs/heads/main/Dados/Telco-Customer-Churn.json"

response = requests.get(url)
data = json.loads(response.text)

# Normalizando o JSON aninhado para um DataFrame
df_raw = pd.json_normalize(data)

print("Dados extraídos com sucesso!")
df_raw.head()

### Transformação e Limpeza

Após a extração, os dados brutos passaram por um processo de transformação e limpeza para garantir sua qualidade e usabilidade. As principais etapas foram:

1.  **Renomeação das Colunas:** As colunas continham prefixos (ex: `customer.gender`). Esses prefixos foram removidos para simplificar o acesso aos dados.
2.  **Tratamento de Dados Numéricos:** A coluna `account.Charges.Total` estava formatada como texto (`object`). Foi necessário convertê-la para um formato numérico (`float`). Durante essa conversão, identificamos valores em branco, que foram tratados.
3.  **Tratamento de Valores Ausentes:** A conversão da coluna de cobrança total gerou 11 valores nulos. Uma investigação revelou que esses valores correspondiam a clientes com `tenure` (meses de contrato) igual a zero. Portanto, os valores nulos foram preenchidos com 0.
4.  **Tradução de Conteúdo:** Para facilitar a compreensão por uma audiência falante de português, os nomes das colunas e os valores categóricos (como 'Yes'/'No', 'Male'/'Female') foram traduzidos.


In [ ]:
# 1. Renomear colunas
df_cleaned = df_raw.copy()
df_cleaned.columns = [col.split('.')[-1] if '.' in col else col for col in df_cleaned.columns]

# 2. Tratar a coluna 'Total'
df_cleaned['Total'] = pd.to_numeric(df_cleaned['Total'], errors='coerce')

# 3. Preencher valores nulos
df_cleaned['Total'] = df_cleaned['Total'].fillna(0)

# 4. Remover linhas com 'Churn' vazio (se houver)
df_cleaned = df_cleaned[df_cleaned['Churn'] != '']

# 5. Traduzir colunas e valores
traducao_colunas = {
    'customerID': 'ID_Cliente',
    'Churn': 'Churn',
    'gender': 'Genero',
    'SeniorCitizen': 'Idoso',
    'Partner': 'Parceiro',
    'Dependents': 'Dependentes',
    'tenure': 'Meses_Contrato',
    'PhoneService': 'Servico_Telefonico',
    'MultipleLines': 'Multiplas_Linhas',
    'InternetService': 'Servico_Internet',
    'OnlineSecurity': 'Seguranca_Online',
    'OnlineBackup': 'Backup_Online',
    'DeviceProtection': 'Protecao_Dispositivo',
    'TechSupport': 'Suporte_Tecnico',
    'StreamingTV': 'Streaming_TV',
    'StreamingMovies': 'Streaming_Filmes',
    'Contract': 'Contrato',
    'PaperlessBilling': 'Fatura_Digital',
    'PaymentMethod': 'Metodo_Pagamento',
    'Monthly': 'Valor_Mensal',
    'Total': 'Valor_Total'
}
df_cleaned.rename(columns=traducao_colunas, inplace=True)

mapeamento_sim_nao = {'Yes': 'Sim', 'No': 'Não', 'No internet service': 'Sem serviço de internet', 'No phone service': 'Sem serviço telefônico'}
for col in ['Churn', 'Parceiro', 'Dependentes', 'Servico_Telefonico', 'Multiplas_Linhas', 'Seguranca_Online', 'Backup_Online', 'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV', 'Streaming_Filmes', 'Fatura_Digital']:
    df_cleaned[col] = df_cleaned[col].replace(mapeamento_sim_nao)

df_cleaned['Genero'] = df_cleaned['Genero'].replace({'Female': 'Feminino', 'Male': 'Masculino'})

print("Dados limpos e transformados com sucesso!")
df_cleaned.info()

## 2. Análise Exploratória de Dados (EDA)

Com os dados limpos, iniciamos a análise exploratória para identificar padrões e correlações relacionadas ao churn. A taxa de churn geral na base de dados é de **26.54%**.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.countplot(data=df_cleaned, x='Churn', palette='viridis')
plt.title('Distribuição de Churn (Evasão de Clientes)')
plt.show()

### Churn por Tipo de Contrato

A análise revela que o tipo de contrato é um dos fatores mais influentes no churn. Clientes com contratos mensais (`Month-to-month`) têm uma taxa de churn drasticamente maior.

- **Contrato Mensal:** 42.71% de churn
- **Contrato de Um Ano:** 11.27% de churn
- **Contrato de Dois Anos:** 2.83% de churn

Isso sugere que clientes com compromissos de longo prazo são muito mais propensos a permanecer na empresa.


In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_cleaned, x='Contrato', hue='Churn', palette='Set2')
plt.title('Churn por Tipo de Contrato')
plt.show()

### Churn por Serviço de Internet

O tipo de serviço de internet também mostra uma correlação forte com a evasão.

- **Fibra Óptica:** 41.89% de churn
- **DSL:** 18.96% de churn
- **Sem Internet:** 7.40% de churn

Clientes com **Fibra Óptica** apresentam a maior taxa de churn. Embora possa parecer contraintuitivo (já que é um serviço premium), isso pode indicar problemas de instabilidade, preço ou uma percepção de baixo valor agregado em comparação com a concorrência para este serviço específico.


In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df_cleaned, x='Servico_Internet', hue='Churn', palette='coolwarm')
plt.title('Churn por Serviço de Internet')
plt.show()

### Churn por Cobranças Mensais e Tempo de Contrato

- **Cobranças Mensais:** Clientes que cancelaram tendem a ter cobranças mensais mais altas, como visto na densidade do gráfico abaixo.
- **Tempo de Contrato (Tenure):** Clientes com poucos meses de contrato são muito mais propensos a cancelar. A base de clientes fiéis (baixo churn) é composta majoritariamente por clientes com longo tempo de casa.


In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(data=df_cleaned, x='Valor_Mensal', hue='Churn', fill=True, common_norm=False, palette='crest')
plt.title('Distribuição de Valor Mensal por Churn')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_cleaned, x='Churn', y='Meses_Contrato', palette='pastel')
plt.title('Meses de Contrato (Tenure) vs Churn')
plt.show()

## 3. Conclusão e Recomendações

A análise exploratória revelou padrões claros associados à evasão de clientes na Telecom X. Os fatores de maior impacto são o **tipo de contrato**, o **tempo de permanência (tenure)** e o **tipo de serviço de internet**.

### Principais Achados:

1.  **Vulnerabilidade em Contratos Mensais:** Clientes sem um vínculo de longo prazo representam o maior risco de churn. A flexibilidade do contrato mensal facilita o cancelamento.
2.  **Insatisfação com Fibra Óptica:** A alta taxa de churn entre usuários de fibra óptica é um sinal de alerta. A causa precisa (preço, qualidade, suporte) necessita de uma investigação mais aprofundada.
3.  **Risco Elevado em Novos Clientes:** A evasão é significativamente maior nos primeiros meses de serviço, indicando possíveis falhas no processo de onboarding ou desalinhamento de expectativas.

### Recomendações para a Equipe de Data Science:

- **Modelagem Preditiva:** Utilizar as variáveis `Contrato`, `Meses_Contrato`, `Servico_Internet` e `Valor_Mensal` como features importantes para os modelos de previsão de churn.
- **Segmentação de Clientes:** Criar segmentos de risco com base nesses fatores. Por exemplo, um cliente com contrato mensal, serviço de fibra e poucos meses de contrato deve ser classificado como de altíssimo risco.

### Sugestões para a Empresa:

- **Incentivos para Contratos de Longo Prazo:** Oferecer descontos ou benefícios mais agressivos para clientes que migrarem de contratos mensais para anuais ou bianuais.
- **Investigar a Qualidade do Serviço de Fibra:** Realizar uma análise de causa raiz sobre a insatisfação dos clientes de fibra. Isso pode envolver pesquisas de satisfação, análise de chamados de suporte técnico e comparação com concorrentes.
- **Programa de Fidelidade para Novos Clientes:** Criar um programa de acompanhamento e benefícios para clientes nos primeiros 3 a 6 meses, garantindo uma experiência positiva inicial e fortalecendo o relacionamento.

Este relatório fornece uma base sólida para as próximas etapas do projeto. Os dados tratados e os insights gerados permitirão que a equipe de Data Science construa modelos preditivos mais precisos e que a Telecom X tome decisões mais estratégicas para reduzir a evasão de clientes.
